# Week 9 - Assessed exercises

In this weeks exercises we will fit a regression model and create a stepwise AIC function.

Submit your answers to the questions below on Moodle.

In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import numpy.random as npr
import statsmodels.api as sm

In this week's assessed exercises we will look at the prostate data set, which we used in this week's material. 

Load in the prostate dataset.
Create a Series y which contains the response variable (lpsa).

In [26]:
prostate = pd.read_csv("prostate.csv")
y = prostate.loc[:,["lpsa"]]
y

,lpsa
0,-0.430783
1,-0.162519
2,-0.162519
3,-0.162519
4,0.371564
...,...
92,4.385147
93,4.684443
94,5.143124
95,5.477509


Create a DataFrame X which contains the explanatory variables (lcavol, lweight, age, lbph, svi, lcp, gleason, and pgg45). Standardise X, such that all variables have mean 0 and standard deviation 1.

In [75]:
x = prostate.loc[:,["lcavol","lweight","age","lbph","svi","lcp","gleason","pgg45"]]
x = (x - x.mean()) / x.std()

Add an intercept column to X. The intercept column should be the first column of X.

In [76]:
intercept = pd.DataFrame({"intercept":np.ones(97)})
x = intercept.merge(x,right_index=True,left_index=True)
prostate_std = y.merge(x,right_index=True,left_index=True)

Which column of X has the smallest correlation with y?

In [77]:
prostate_std.corr()

,lpsa,intercept,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lpsa,1.000000,NaN,0.734460,0.433319,0.169593,0.179809,0.566218,0.548813,0.368987,0.422316
intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lcavol,0.734460,NaN,1.000000,0.280521,0.225000,0.027350,0.538845,0.675310,0.432417,0.433652
lweight,0.433319,NaN,0.280521,1.000000,0.347969,0.442264,0.155385,0.164537,0.056882,0.107354
age,0.169593,NaN,0.225000,0.347969,1.000000,0.350186,0.117658,0.127668,0.268892,0.276112
lbph,0.179809,NaN,0.027350,0.442264,0.350186,1.000000,-0.085843,-0.006999,0.077820,0.078460
svi,0.566218,NaN,0.538845,0.155385,0.117658,-0.085843,1.000000,0.673111,0.320412,0.457648
lcp,0.548813,NaN,0.675310,0.164537,0.127668,-0.006999,0.673111,1.000000,0.514830,0.631528
gleason,0.368987,NaN,0.432417,0.056882,0.268892,0.077820,0.320412,0.514830,1.000000,0.751905
pgg45,0.422316,NaN,0.433652,0.107354,0.276112,0.078460,0.457648,0.631528,0.751905,1.000000


Use the `OLS` function from statsmodels.api to fit a linear regression with y as your dependent/response variable and the first two columns of X as the explanatory variables, i.e. the intercept column and the lcavol column. 

What is the adjusted R-squared to 3 decimal places?

In [142]:
model = sm.OLS(y.lpsa,x.iloc[:,0:2])
res = model.fit()
np.round(res.rsquared_adj,decimals=3)

x

,intercept,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
0,1.0,-1.637356,-2.006212,-1.862426,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467
1,1.0,-1.988980,-0.722009,-0.787896,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467
2,1.0,-1.578819,-2.188784,1.361163,-1.024706,-0.522941,-0.863171,0.342627,-0.155348
3,1.0,-2.166917,-0.807994,-0.787896,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467
4,1.0,-0.507874,-0.458834,-0.250631,-1.024706,-0.522941,-0.863171,-1.042157,-0.864467
...,...,...,...,...,...,...,...,...,...
92,1.0,1.255920,0.577607,0.555266,-1.024706,1.892548,1.073572,0.342627,1.262889
93,1.0,2.096506,0.625489,-2.668323,-1.024706,1.892548,1.679542,0.342627,0.553770
94,1.0,1.321402,-0.543304,-1.593794,-1.024706,1.892548,1.890377,0.342627,-0.509907
95,1.0,1.300290,0.338384,0.555266,1.004813,1.892548,1.242632,0.342627,1.972007


We now want to run a *forward selection AIC regression*. AIC is the Akaike information criterion. It's designed to penalise models with lots of explanatory variables so that we pick models which fit the data well but aren't too complicated. In general, if you have two models fitted to the same data, the model with the lowest AIC is preferable. The AIC is given as part of the model summary with OLS.

The steps to run a forward selection AIC regression are: 
1. Begin with a model that contains no variable (other than the intercept). Run a linear regression and record the AIC. For now, this is our *current model*.
2. Find the most significant variable, i.e. the variable that lowers the AIC the most <br>
a.  Run a linear regression with the *current model* plus one additional variable, and record the decrease in AIC.<br>
b. Repeat step 2a for each variable not included in the *current model*. <br>
c. Find the variable with the biggest decrease in AIC. <br>
d. Update the *current model* to include the variable that decreases the AIC the most.
3. If none of the variables lower the AIC then go to step 4. Otherwise repeat step 2 until adding variables no longer reduces the AIC.
4. Report your final chosen variables

Write a function called `forwardAIC` which performs this algorithm given the DataFrame X and Series y.
The function should return the column numbers of the X matrix for the model that gives the lowest AIC.

In [143]:
def forwardAIC(y,x):
    model = sm.OLS(y.lpsa,x.iloc[:,0])
    res = model.fit()
    
    current_columns = ["intercept"]
    all_columns = x.columns

    base_aic = res.aic
    cols = []

    while True:
        best_column = ""
        best_aic = base_aic
        for i in range(len(all_columns)):
            if(all_columns[i] not in current_columns):
                cols = [*current_columns,all_columns[i]]
                model = sm.OLS(y.lpsa,x.loc[:,cols]).fit()
                aic = model.aic

                if(aic < best_aic):
                    best_aic = aic
                    best_column = all_columns[i]
                print("AIC = ", aic,end="   ")
                print("Current column =", all_columns[i], end="   ")
                print("Selected columns =", cols,end="\n")
        
        print("\n\n\n")
        if(best_column == ""): break
        current_columns.append(best_column)
        base_aic = best_aic
    print(best_aic)
    return current_columns
forwardAIC(y,x)

AIC =  230.90804052077908   Current column = lcavol   Selected columns = ['intercept', 'lcavol']
AIC =  285.93888438200486   Current column = lweight   Selected columns = ['intercept', 'lweight']
AIC =  303.28083171633   Current column = age   Selected columns = ['intercept', 'age']
AIC =  302.92366098990567   Current column = lbph   Selected columns = ['intercept', 'lbph']
AIC =  268.61630117756584   Current column = svi   Selected columns = ['intercept', 'svi']
AIC =  271.3482979645054   Current column = lcp   Selected columns = ['intercept', 'lcp']
AIC =  291.9149398511339   Current column = gleason   Selected columns = ['intercept', 'gleason']
AIC =  287.05679195458356   Current column = pgg45   Selected columns = ['intercept', 'pgg45']




AIC =  220.31561400279492   Current column = lweight   Selected columns = ['intercept', 'lcavol', 'lweight']
AIC =  232.90386318146312   Current column = age   Selected columns = ['intercept', 'lcavol', 'age']
AIC =  227.3763524550538   Current 

['intercept', 'lcavol', 'lweight', 'svi', 'lbph', 'age']


Which five variables (not including the constant) come back as important?

In [ ]:
['intercept', 'lcavol', 'lweight', 'svi', 'lbph', 'age']

304.1116269220838


What's the AIC of this chosen model?

In [ ]:
211.55144411323568

**Bonus question (ungraded)**

(Included in the non-assessed exercises on Moodle, if you wish to check your answers)

Run the same analysis on the full Diamonds data (given with this notebook) using price as the dependent/response variable. Load the data in and create dummy variables for the categorical variables cut, colour and clarity (using `pd.get_dummies`). You will need to drop one category for each categorical variable (i.e. drop 'Fair' for cut, drop 'D' for color, and drop 'I1' for clarity). Otherwise the model cannot be fully determined.


How many variables (not including the constant) get chosen?

What's the AIC of this chosen model?